###  Install requirements :

CrewAI is available as a library or with extensions. Here, we’ll be using some extensions, such as Tools and AgentOps, for monitoring agent operations.

In [2]:
!pip install -qU crewai[tools,agentops]==0.95.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [55]:
!pip install -qU tavily-python scrapegraph-py

### Import libraries :

In [75]:
from crewai import Agent, Task, Crew, Process, LLM
import agentops
from google.colab import userdata
import os
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from crewai.tools import tool
from scrapegraph_py import Client
import json
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

### Set up OpenAI and Agentops :

In [45]:
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["AGENTOPS_API_KEY"] = userdata.get('agentops_key')

agentops.init(
    api_key = userdata.get('agentops_key'),
    skip_auto_end_session = True      # Keep the session alive; don't end automatically
)

### Create a directory to save docs:

In [46]:
output_dir = "./ai_agent_output"
os.makedirs(output_dir, exist_ok=True)

### Set basic LLM :

In [47]:
basic_llm = LLM(model="gpt-4o", temperature=0)

In [48]:
no_keywords = 10

### Define a tavily client :

In [57]:
search_client = TavilyClient(api_key=userdata.get('tavily'))

In [58]:
# Test :
search_client.search("coffee machine site: amazon.com")

{'query': 'coffee machine site: amazon.com',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.amazon.com/stores/Cuisinart/page/F5136BA1-5C29-40B7-8B52-2E5CF6492330',
   'title': 'Cuisinart: Coffee Makers - Amazon.com',
   'content': "Enjoy perfectly brewed coffee every cup with Cuisinart's affordable deluxe coffeemakers.",
   'score': 0.6649108,
   'raw_content': None},
  {'url': 'https://www.amazon.com/Coffee-Makers/b?ie=UTF8&node=7740213011',
   'title': 'Amazon.com: Coffee Makers',
   'content': '30-day returnsDiscover Coffee Makers on Amazon.com at a great price. Our Coffee, Tea & Espresso category offers a great selection of Coffee Makers and more.',
   'score': 0.6551821,
   'raw_content': None},
  {'url': 'https://www.amazon.com/Drip-Coffee-Machines-Makers/b?ie=UTF8&node=289745',
   'title': 'Amazon.com: Coffee Machines',
   'content': '## Hamilton Beach 2-Way 12 Cup Programmable Drip Coffee Maker & Single Serve Machine, Glass Caraf

### Define a scrape graph client :

In [65]:
scrape_client = Client(api_key=userdata.get('scrape_graph_ai'))

### Create a knowledge source :

In [77]:
about_company = "RankX is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

## Set up Agents :

### Agent A :

* **Role:** Research Initiator
* **Task:** Generate a list of suggested queries (keywords) to feed into the search engine.
* **Tools:** Native keyword extraction skills.
* **Output:** Keywords list.


In [49]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items=1, max_items=no_keywords)



search_queries_recommendation_AGENT = Agent(
    role = "Search Queries Recommendation Agent.",
    goal = "\n".join([
                "To provide a list of suggested search queries to be passed to the search engine.",
                "The queries must be varied and looking for specific items."
            ]),
    backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
)



search_queries_recommendation_TASK = Task(
     description="\n".join([
        "RankX is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The campany target any of these websites to buy from: {websites_list}",
        "The company wants to reach all available proucts on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "The search keywords must be in {language} language.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ]),
     expected_output="A JSON object containing a list of suggested search queries.",
     output_json=SuggestedSearchQueries,
     output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
     agent=search_queries_recommendation_AGENT
)

### Agent B :

* **Role:** Online Searcher
* **Task:** Perform web searches for products using the keywords.
* **Tools:** https://tavily.com a free alternative for Google Search API (paid)
* **Output:** List of relevant web results.

In [61]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str



class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]


@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)


search_engine_AGENT = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)


search_engine_TASK = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any susbicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th}) .",
        "The search results will be used to compare prices of products from different websites.",
    ]),
    expected_output="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_AGENT
)

### Agent C :

* **Role:** Information Miner
* **Task:** Parse and extract product details (pricing, specs, supplier info) from web pages.
* **Tools:** https://scrapegraphai.com or scraping script by selenuim or scrapy
* **Output:** Structured product details.

In [68]:
# Test
def web_scraping_tool(page_url: str, required_fields: list):
    """
    An AI Tool to help an agent to scrape a web page

    Example:
    web_scraping_tool(
        page_url="https://www.noon.com/egypt-en/15-bar-fully-automatic-espresso-machine-1-8-l-1500"
    )
    """
    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt="Extract "+ json.dumps(required_fields, ensure_ascii=False) + "From the web page."
    )

    return {
        "page_url": page_url,
        "details": details
    }

In [69]:
page_url = "https://www.amazon.com/Daikin-17-5-Single-Ductless-Conditioner/dp/B07JMGGYLC"
required_fields = ["product_title", "current_price"]
page_details = web_scraping_tool(page_url, required_fields)

In [70]:
page_details

{'page_url': 'https://www.amazon.com/Daikin-17-5-Single-Ductless-Conditioner/dp/B07JMGGYLC',
 'details': {'request_id': 'da53619c-cc11-4a66-8d6d-98a37052b97d',
  'status': 'completed',
  'website_url': 'https://www.amazon.com/Daikin-17-5-Single-Ductless-Conditioner/dp/B07JMGGYLC',
  'user_prompt': 'Extract ["product_title", "current_price"]From the web page.',
  'result': {'product_title': 'DAIKIN 30,000 Btu 17.5 Seer Single Zone Ductless Air Conditioner',
   'current_price': "Currently unavailable. We don't know when or if this item will be back in stock."},
  'error': ''}}

In [71]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)

    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]


@tool
def web_scraping_tool(page_url: str):
    """
    An AI Tool to help an agent to scrape a web page
    """
    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt="Extract ```json\n" + SingleExtractedProduct.schema_json() + "```\n From the web page"
    )

    return {
        "page_url": page_url,
        "details": details
    }


scraping_AGENT = Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy.",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=True,
)


scraping_TASK = Task(
    description="\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the best {top_recommendations_no} products from the search results.",
    ]),
    expected_output="A JSON object containing products details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    agent=scraping_AGENT
)

### Agent D : reporting

* **Role:** Procurement Analyst
* **Task:** Transform structured product details into a professional, human-readable procurement report.
* **Tools:** HTML/PDF report generator.
* **Output:** Visual, ready-to-share procurement report.


In [78]:
procurement_report_author_AGENT = Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True,
)

procurement_report_author_TASK = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=procurement_report_author_AGENT,
)

## Run the AI crew :

In [79]:

RankX_crew = Crew(

    agents=[
        search_queries_recommendation_AGENT,
        search_engine_AGENT,
        scraping_AGENT,
        procurement_report_author_AGENT,
    ],

    tasks = [
        search_queries_recommendation_TASK,
        search_engine_TASK,
        scraping_TASK,
        procurement_report_author_TASK,
    ],

    process=Process.sequential,
    knowledge_sources=[company_context]

)

In [80]:
# Execute the crew :
crew_results = RankX_crew.kickoff(
      inputs={
          "product_name": "Office air conditioners",
          "websites_list": ["www.amazon.com", "www.electroplanet.ma", "www.ikea.com"],
          "country_name": "Morocco",
          "no_keywords": 10,
          "language": "English",
          "score_th": 0.10,
          "top_recommendations_no": 10
      },
  )

╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Search Queries Recommendation Agent.                                 │
│                                                                              │
│  Task: RankX is looking to buy Office air conditioners at the best prices    │
│  (value for a price strategy)                                                │
│  The campany target any of these websites to buy from: ['www.amazon.com',    │
│  'www.electroplanet.ma', 'www.ikea.com']                                     │
│  The company wants to reach all available proucts on the internet to be      │
│  compared later in another stage.                                            │
│  The stores must sell the product in Morocco                                 │
│  Generate at maximum 10 queries.                                             │
│  The search keywords must b

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I now ca...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I will s...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [

╭────────────────────────── 🔧 Agent Tool Execution ───────────────────────────╮
│                                                                              │
│  Agent: Search Engine Agent                                                  │
│                                                                              │
│  Thought: I will start by searching for products based on the provided       │
│  queries using the search engine tool. I will go through each query one by   │
│  one and collect the results.                                                │
│  Let's begin with the first query: "Daikin office air conditioner            │
│  site:www.amazon.com".                                                       │
│                                                                              │
│  Using Tool: search_engine_tool                                              │
│                                                                              │
╰────────────────────────────

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I will n...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


╭────────────────────────── 🔧 Agent Tool Execution ───────────────────────────╮
│                                                                              │
│  Agent: Search Engine Agent                                                  │
│                                                                              │
│  Thought: I will now proceed with the next query: "LG inverter air           │
│  conditioner for office site:www.electroplanet.ma".                          │
│                                                                              │
│  Using Tool: search_engine_tool                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
╭───────────────────────────────── Tool Input ─────────────────────────────────╮
│                                                                              │
│  "{\"query\": \"LG inverter

...

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I have s...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [

╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Procurement Report Author Agent                                      │
│                                                                              │
│  Task: The task is to generate a professional HTML page for the procurement  │
│  report.                                                                     │
│  You have to use Bootstrap CSS framework for a better UI.                    │
│  Use the provided context about the company to make a specialized report.    │
│  The report will include the search results and prices of products from      │
│  different websites.                                                         │
│  The report should be structured with the following sections:                │
│  1. Executive Summary: A brief overview of the procurement process and key   │
│  findings.                 

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


╭─────────────────────────── ✅ Agent Final Answer ────────────────────────────╮
│                                                                              │
│  Agent: Procurement Report Author Agent                                      │
│                                                                              │
│  Final Answer:                                                               │
│  ```html                                                                     │
│  <!DOCTYPE html>                                                             │
│  <html lang="en">                                                            │
│  <head>                                                                      │
│      <meta charset="UTF-8">                                                  │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">  │
│      <title>Procurement Report</title>                                       │
│      <link                 

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
